# Dependencies

In [1]:
import MURA
import cv2
import image_manipulation
from multiprocessing import Pool
from models import *
import numpy as np
import glob
import matplotlib.pyplot as plt
import preprocessing
import json
from os import path, mkdir

import keras
from argparse import ArgumentParser
import tensorflow as tf

2023-05-20 12:11:49.251845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-20 12:11:49.336187: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-20 12:11:49.338071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 12:11:50.356654: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/cara/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model to Run
This section lets the user edit on what model parameters to run, the model directory and parameters should exist prior to changing the file path

In [2]:
# All editable variables
model_file_path = "models/model_1"
# If preprocessing needs to run
run_preprocessing = False

In [3]:
# Get model parameters
try:
    params = json.load(open(model_file_path + '/parameters.json'))

    # Model to use
    model_is_VAE = params['is_VAE']
    # Model parameters
    multiplier = params['multiplier']
    latent_size = params['latent_size']
    input_shape = params['input_shape']

    # Training parameters
    epochs = params['num_epochs']
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']

    # Dataset Path
    image_paths = MURA.MURA_DATASET()
    dataset_file_path = params['dataset_path']
    all_image_paths = image_paths.get_combined_image_paths()
    all_image_paths = all_image_paths.to_numpy()[:,0]
except:
    raise Exception("No parameters.json file found in the model's directory.")

In [4]:
# Do preprocessing
if run_preprocessing:
    preprocess = preprocessing.preprocessing(input_path = all_image_paths, output_path = dataset_file_path)
    if __name__ == '__main__':
        preprocess.start()

In [5]:
# each array contains the training, validation, and testing in order
image_datasets = {'train': [],
                'valid': [],
                'test': []}
for dataset_name in image_datasets.keys():
    for image_path in glob.glob(f'{dataset_file_path}/{dataset_name}/*.png'):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_datasets[dataset_name].append(image)
    image_datasets[dataset_name] = np.array(image_datasets[dataset_name])

# Model Training
This section creates and trains the model

In [6]:
if __name__ == "__main__":

    if model_is_VAE:
        model = VAE(False, input_shape, multiplier, latent_size)
    else:
        model = UPAE(True, input_shape, multiplier, latent_size)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.build(input_shape=(None,) + tuple(input_shape))

    model.compile(optimizer= optimizer
                  ,metrics=[tf.keras.metrics.Accuracy()])
    
    model.summary()
    # Where images of each epoch will be saved
    # save_directory = 'Images/images_epochs' #edited in models where automatically make folder if non existent
    save_callback = SaveImageCallback(image_datasets['train'])

    # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
    #training on training set.
    history_train = model.fit(image_datasets['train'], 
                epochs=epochs, 
                batch_size=batch_size,
                validation_split=0.15,
                callbacks=[save_callback])
    

2023-05-20 12:11:56.576392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


INPUT SHAPE ACCEPTED:  (None, 64, 64, 1)
Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_decoder (encoder_de  multiple                 19946433  
 coder)                                                          
                                                                 
Total params: 19,946,439
Trainable params: 19,935,937
Non-trainable params: 10,502
_________________________________________________________________
Epoch 1/3
Vanilla Loss
Data Shape:  (None, 64, 64)
Reconstructed Shape:  (None, 64, 64)
Vanilla Loss
Data Shape:  (None, 64, 64)
Reconstructed Shape:  (None, 64, 64)
38/38 [==============================] - ETA: 0s - mse_loss: 5677.8623 - reconstruction_loss: -412.7689 - kl_loss: 0.0000e+004
callback predict
38/38 [==============================] - 40s 1s/step - mse_loss: 5677.8623 - reconstruction_loss: -412.7689 - kl_loss: 0.0000e+00 - val_mse_loss: 0.0000e+00 

# Model Prediction

In [8]:
history_valid = model.predict(image_datasets['test'], batch_size=batch_size)

not callback predict


KeyboardInterrupt: 

In [13]:
history_valid[1]

[<tf.Tensor: shape=(), dtype=float32, numpy=526.3887>,
 <tf.Tensor: shape=(), dtype=float32, numpy=503.33307>,
 <tf.Tensor: shape=(), dtype=float32, numpy=518.7224>,
 <tf.Tensor: shape=(), dtype=float32, numpy=545.61206>,
 <tf.Tensor: shape=(), dtype=float32, numpy=553.5244>,
 <tf.Tensor: shape=(), dtype=float32, numpy=306.3176>,
 <tf.Tensor: shape=(), dtype=float32, numpy=304.89743>,
 <tf.Tensor: shape=(), dtype=float32, numpy=420.953>,
 <tf.Tensor: shape=(), dtype=float32, numpy=285.70914>,
 <tf.Tensor: shape=(), dtype=float32, numpy=298.20078>,
 <tf.Tensor: shape=(), dtype=float32, numpy=278.27545>,
 <tf.Tensor: shape=(), dtype=float32, numpy=286.6841>,
 <tf.Tensor: shape=(), dtype=float32, numpy=282.879>,
 <tf.Tensor: shape=(), dtype=float32, numpy=285.29175>,
 <tf.Tensor: shape=(), dtype=float32, numpy=232.73627>,
 <tf.Tensor: shape=(), dtype=float32, numpy=189.88045>,
 <tf.Tensor: shape=(), dtype=float32, numpy=692.7303>,
 <tf.Tensor: shape=(), dtype=float32, numpy=711.26746>,
 <

# Plots Creation

In [ ]:
# train_loss = history_train.history['binary_crossentropy: ']
# # valid_loss = history_valid.history['binary_crossentropy: ']

# plt.plot(train_loss , label='train')
# # plt.plot(valid_loss , label='test')
# plt.title('Train vs Validation Loss')
# plt.ylabel('Reconstruction Loss')
# plt.xlabel('Epochs')
# plt.legend()
# plt.show()

# Testing of the Model with the Test Set
This section tests the model with the current test set
TODO: 
- Get the label of each image in the test set
- Test the images
- Create Linear Regression for the abnormality score to get the threshold for determining abnormal or normal images

# Saving of final reconstructed images 

In [9]:
# Create directory in models folder for reconstructed images
dataset_name = dataset_file_path.split('/')[-1]
reconstructed_images_path = model_file_path + "/" + dataset_name
if not path.exists(reconstructed_images_path):
    mkdir(reconstructed_images_path)

In [14]:
for x in range(len(history_valid)):
    fig, axs = plt.subplots(1,2, figsize=(8,4))
    axs[0].imshow(image_datasets['test'][x])
    axs[0].set_title('Original Image')
    new_image = np.floor(history_valid[0][x]).astype(np.uint8)
    axs[1].imshow(new_image)
    axs[1].set_title('Reconstructed Image')
    plt.savefig(f'{reconstructed_images_path}/Valid_Image_{x}.png')
    plt.close()
    break

# Saving of Model Weights

In [ ]:
model.save_weights(model_file_path + '/model_weights.h5')

In [ ]:
history_valid[0].shape

In [ ]:
new_image = np.concatenate(history_valid[0], axis=1)
plt.imshow(new_image)

In [ ]:
history_valid[0]